# Online classifier comparison

This notebook takes a single epoch from either the MOABB or the pediatric dataset and computes the classification running a preset number of voters. The execution time is measured.
The end part of the notebook can do the plot of the accuracy vs number voters.

## Import libraries


In [1]:
# Default libraries
import re
import mne
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

from moabb.datasets import Wang2016, SSVEPExo
from moabb.paradigms import FilterBankSSVEP, SSVEP
from moabb.pipelines import ExtendedSSVEPSignal
from sklearn.linear_model import LogisticRegression
from moabb.evaluations import CrossSubjectEvaluation

import time
from cupyx.profiler import benchmark

# Import custom libraries
from functions import data_tools
from functions import processing
from functions import classification
from functions.FeatureExtractorSSVEP import FeatureExtractorCCA as CCA
from functions.FeatureExtractorSSVEP import FeatureExtractorMSI as MSI
from functions.FeatureExtractorSSVEP import FeatureExtractorMEC as MEC

# Magic command to reload libraries
%reload_ext autoreload

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
c:\Users\danie\miniconda3\envs\ssvep-validation\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
c:\Users\danie\miniconda3\envs\ssvep-validation\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path 

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


## General settings

In [2]:
# Create linear distribution of voters
n_voters = np.linspace(1, 100, 5, dtype=int)
n_epochs = 2   # Number of epochs to be used in the classification


test_dataset = "Wang2016"    # "Wang2016" or "BCI4Kids"
dataset_nsubjects = {
    "Wang2016": 34,
    "BCI4Kids": 6
}
# Generate random indexes
random_seed = 42
# random.seed(random_seed)
# subject = random.randint(0, dataset_nsubjects[dataset] - 1, random_seed)


# Classifier settings 
harmonic_count = 2

# Create dataframes for each classifier
columns = {
    'n_voters': n_voters,
    'CPU_time': np.nan,
    'CPU_accuracy': np.nan,
    'GPU_time': np.nan,
    'GPU_accuracy': np.nan
    }
cca_df = pd.DataFrame(columns)
# msi_df = pd.DataFrame(index=n_voters, columns=['CPU_time', 'CPU_prediction', 'GPU_time', 'GPU_prediction'])
# mec_df = pd.DataFrame(index=n_voters, columns=['CPU_time', 'CPU_prediction', 'GPU_time', 'GPU_prediction'])
# rg_logreg_df = pd.DataFrame(index=n_voters, columns=['CPU_time', 'CPU_prediction', 'GPU_time', 'GPU_prediction'])

# Data

Get a randomized single epoch from a single trial in the selected dataset

In [3]:
if (test_dataset == "Wang2016"):
    # Get trial data
    dataset = Wang2016()
    subject_data = dataset.get_data(subjects=[1])
    time_trim = [0.5, 5.5] # Times to trim the data [sec]
    stimulus_freqs = [float(freq) for freq in dataset.event_id.keys()]  # Stimulus frequencies [Hz]
    srate = subject_data[1]["0"]["0"].info['sfreq'] # Sampling rate [Hz]

    # Frequency bands
    first_column = np.arange(1, 11) * 8
    second_column = np.full(10, 88)
    cca_subbands = np.column_stack((first_column, second_column))

    # Get epoched data
    # - Preallocate data
    epochs_list = [None] * len(subject_data)
    events_list = [None] * len(subject_data)

    # - Obtain epochs and events
    for s, subject in subject_data.items():
        [events_list[s-1], epochs_list[s-1]] = data_tools.moabb_events_to_np(
            mne_raw = subject["0"]["0"],
            tmin = time_trim[0],
            tmax = time_trim[1],
            events_dict = dataset.event_id,
            )
        
    # - Convert lists to np.ndarrays
    eeg_channels = subject_data[1]["0"]["0"].ch_names
    epochs_np = np.float32(np.array(epochs_list))
    events_np = np.array(events_list[0][:,2]) - 1   # The `-1` is to make the labels start at 0

    # - Remove stim channel and pick subset of epochs
    eeg_channels = eeg_channels[:-1]
    epochs_np = epochs_np[:, :, :-1, :]
    n_samples = epochs_np.shape[-1]

    # - Get randomized subset of epochs
    total_n_epochs = epochs_np.shape[1]
    epochs_subset = random.sample(range(total_n_epochs), n_epochs)
    # epochs_subset = 0
    epochs = epochs_np[:,epochs_subset,:,:]



Trial data de-meaned and concatenated with a buffer to create continuous data


# Setup classifiers

In [4]:
cca = CCA()
cca_voters = 1
cca_gpu = False

def setup_cca():
    cca.setup_feature_extractor(
        harmonics_count = harmonic_count,
        targets_frequencies = stimulus_freqs,
        sampling_frequency = srate,
        samples_count = n_samples,
        filter_order = 12,
        subbands = cca_subbands,
        voters_count = cca_voters,
        use_gpu = cca_gpu,
        random_seed = random_seed
        )
    
mec = MEC()
def setup_mec(voters, use_gpu):
    # Missing implementation
    pass

msi = MSI()
def setup_msi(voters, use_gpu):
    # Missing implementation
    pass

## CPU classification

### CCA

In [5]:
# Classify epoch with varying number of voters
print("CCA CPU processing")
for (v, voters) in enumerate(n_voters):
    temp_times = np.zeros(n_epochs)
    temp_accuracy = 0

    for e, epoch in enumerate(epochs_subset):
        print(f"- voters: {voters}")
        start_time = time.time()

        # Set up classifier and extract features
        cca_voters = voters
        cca_gpu = False
        setup_cca()    
        cca_features = np.squeeze(cca.extract_features(epochs[:,e,:,:]))

        # Find max correlation across subbands and voters
        if (voters == 1):
            temp_prediction = np.argmax(np.max(cca_features, axis=0), axis=0)
        else:
            temp_prediction = np.argmax(np.max(cca_features, axis=(0,1)), axis=0)

        end_time = time.time()
        temp_times[e] = end_time - start_time
        print(f"Elapsed time: {end_time - start_time}")

        temp_accuracy += 1 if temp_prediction == events_np[epoch] else 0

    cca_df.loc[v, "CPU_time"] = temp_times.mean()
    cca_df.loc[v, "CPU_accuracy"] = temp_accuracy / n_epochs

CCA CPU processing
- voters: 1
Elapsed time: 0.3287386894226074
- voters: 1
Elapsed time: 0.3113059997558594
- voters: 25
Elapsed time: 1.0343492031097412
- voters: 25
Elapsed time: 1.0509660243988037
- voters: 50
Elapsed time: 1.9302783012390137
- voters: 50
Elapsed time: 1.934511661529541
- voters: 75
Elapsed time: 2.9365484714508057
- voters: 75
Elapsed time: 2.8635051250457764
- voters: 100
Elapsed time: 3.800689935684204
- voters: 100
Elapsed time: 3.805293560028076


## GPU classification

### CCA

In [ ]:
benchmark(
    time_cca_gpu,
    n_repeat = 1,
    n_warmup = 3,
    devices = (0,)
    )

ImportError: DLL load failed while importing cusolver: The specified module could not be found.

In [6]:
def time_cca_gpu():
    for e, epoch in enumerate(epochs_subset):
        setup_cca()    
        cca_features = np.squeeze(cca.extract_features(epochs[:,e,:,:]))

        # Find max correlation across subbands and voters
        if (voters == 1):
            temp_prediction = np.argmax(np.max(cca_features, axis=0), axis=0)
        else:
            temp_prediction = np.argmax(np.max(cca_features, axis=(0,1)), axis=0)

        temp_accuracy += 1 if temp_prediction == events_np[epoch] else 0

    cca_df.loc[v, "CPU_accuracy"] = temp_accuracy / n_epochs

In [ ]:
cca_gpu_results = [None] * len(n_voters)

for (v, voters) in enumerate(n_voters):
    cca_voters = voters
    cca_gpu = True
    temp_accuracy = 0

    for (e, epoch) in enumerate(epochs_subset):
        cca_gpu_results.append(
            benchmark(
                time_cca_gpu,
                n_repeat = 1,
                n_warmup = 3,
                devices = (0,)
                )
            )

ImportError: DLL load failed while importing cusolver: The specified module could not be found.